In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import dataloader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision
from torchvision.transforms import ToTensor

in_channel = 3
number_clases = 10
learning_rate = 1e-3
batch_size = 512
num_epoch = 20

import sys

training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

model = torchvision.models.vgg11(pretrained=False)

for param in model.parameters():
    param.requires_grad = False

model.avgpool = Identity()
model.classifier[0] = nn.Linear(in_features=512, out_features=4096, bias=True)
model.classifier[6] = nn.Linear(in_features=4096, out_features=10, bias=True)
# print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epoch):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):

        scores =model(data)
        loss = criterion (scores, targets)

        losses.append(loss.item())
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

    print(epoch, sum(losses)/len(losses))

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
0 2.270455781294375
1 2.1395199274530214
2 2.0345713046132303
3 1.9625320750839856
4 1.9135165822749236
5 1.8778294641144422
6 1.8485589258524837
7 1.831337000642504
8 1.8075177353255603
9 1.791159069051548
10 1.7799957504077835
11 1.764070182430501
12 1.7519367057449964
13 1.7429955735498546
14 1.729990394747987
15 1.7202793578712308
16 1.7081009611791493
17 1.7006180128272699
18 1.6862329220285222
19 1.679192311909734


In [2]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 39 %


1) Realizar transfer learning a partir de alguna de las siguientes redes: vgg11, resnet18, convnext_tiny preentrenadas, y entrenar sobre el dataset CIFAR10 contenido en PyTorch. Que puede concluir al compararlos con los resultados obtenidos en el entrenamiento de la tarea 2? 

Se concluye una mejor capacidad para afrontar esta tarea por parte de la arquitectura de la tarea 2, la cual obtuvo mejores resultados pese a no estar pre-entrenada.

2) Realice el entrenamiento de la misma red seleccionada en el punto 1, sin partir de pesos preentrenados en esta oportunidad y sin congelar capas. A que resultados llega y como los explica?

Se obtienen peores resultados al clasificar en comparación al modelo pre-entrenado, esto se explica por la necesidad de entrenar una serie de parámetros, los cuales ya vienen aprendidos en la versión pre-entrenada del modelo.
